# Getting Startedを試す

## 概要

[Getting Started](https://automagica.readthedocs.io/#getting-started) では環境構築からExampleの実行まで紹介されている。

サンプルの中でもExample 1では、エクセルに記載された「検索ワード」を読み取り、ChromeブラウザでGoogle検索した後、結果のリンクをエクセルに書き込む、という流れを体験できる。

なお、Example 1ではいくつか都合の悪い個所があったので、以下の実行例では修正しながら実行している。

## 実行

Automagicaでは、automagica以下のメソッドをすべて読み込むことで複数のライブラリにまたがる機能をまとめて利用可能にしてくれる。

In [5]:
from automagica import *
import os

ここではユーザのホームディレクトリ以下 `\Sources.win\automagica_getting_started\test.xlsx` にエクセルを置いたものと仮定する。

In [12]:
excel_path = os.environ["HOMEPATH"] + r"\Sources.win\automagica_getting_started\test.xlsx"

2019/10/29時点の公式ドキュメントのExample 1では、エクセルから検索ワードを拾うときに、for文の上限に足りないときにはリストに `None` が含まれた。これだと後続処理で都合が悪いので、それを除去する処理を追加した。

また2019/10/29時点で公式ドキュメント上では、 `ExcelReadCell` を用いるようになっているが、引数の与え方が誤っている。引数に合わせるならば、`ExcelReadRowCol` を使うべきである。以下の例では `ExcelReadRowCol` を使うようにした。

また2019/10/29時点で公式ドキュメント上では、 row番号に `2` が指定されているが、1行目から検索ワードを読み取る必要があるため、 `1` を指定すべきである。以下の例では `1` を使うようにした。

In [13]:
lookup_terms = []
for col in range(2,10):
    lookup_terms.append(ExcelReadRowCol(excel_path, 1, col))
    
lookup_terms = list(filter(lambda x: x is not None, lookup_terms))

In [14]:
lookup_terms

['Robot', 'President', 'Banana']

Seleniumの機能を利用し、ブラウザを立ち上げる。Automagica APIでは `ChromeBrowser()` で立ち上がる。

In [15]:
browser = ChromeBrowser()

実はAutomagicaには、ChromeのWebDriverが含まれている。上記メソッド内で、以下のように利用するようになっている。

automagica/activities.py:508
```
    if platform.system() == 'Linux':
        chromedriver_path = '\\bin\\webdriver\\linux64\\chromedriver'
    elif platform.system() == 'Windows':
        chromedriver_path = '\\bin\\win32\\chromedriver.exe'
    else:
        chromedriver_path = '\\bin\\mac64\\chromedriver.exe'
```

これを利用し、以下のように立ち上げるようになっている。

automagica/activities.py:515
```
    from selenium.webdriver import Chrome, ChromeOptions

    chrome_options = ChromeOptions()

    if headless:
        chrome_options.add_argument("--headless")

    if ignore_images:
        prefs = {"profile.managed_default_content_settings.images": 2}
        chrome_options.add_experimental_option("prefs", prefs)

    return Chrome(os.path.abspath(__file__).replace('activities.py', '') + chromedriver_path, chrome_options=chrome_options)
```

なお、関数の引数は

automagica/activities.py:491
```
def ChromeBrowser(ignore_images=False, headless=False):
```

となっており、画像無視するかとヘッドあり・なしを選択できるようになっている。

続いて、検索ワードごとにGoogle検索し、結果ページに表示されたリンクを取得してエクセルに書き込む処理は以下の通り。

ここで、読み取り時と同様、公式ドキュメント上ではエクセルへの書き込みに、 `ExcelWriteCell` が使われているが、引数の与え方から見て、 `ExcelWriteRowCol` を用いるべきである。以下の例ではそのように修正してある。

In [16]:
for j,item in enumerate(lookup_terms):

    # Browse to Google
    browser.get('https://google.com')
    # Lookup the searchterm
    browser.find_element_by_xpath('//*[@name="q"]').send_keys(item)
    # Search
    browser.find_element_by_xpath('//*[@name="q"]').submit()
    # Get all found items
    articles = browser.find_elements_by_class_name("g")
    # Parse the headertexts to find the urls
    urls = []
    for article in articles:
        try:
            import re
            urls.append(re.findall('https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', article.text)[0])
        except:
            pass

    # Write found urls to Excel
    for i,url in enumerate(urls):
        ExcelWriteRowCol(excel_path, r=i+2, c=j+2, write_value=url)

最後に、エクセルファイルを開いてどうなったか確認する。

In [18]:
OpenFile(path=excel_path)